In [1]:
# !pip install langchain-community
# !pip install langchain-core
# !pip install tesseract
# !pip install markdownify
# ! pip install semantic-text-splitter
# ! pip install unstructured
# !pip install pytesseract
# !pip install tesseract-ocr
# !pip install docx2txt

In [1]:
import os
import re

from langchain_community.document_loaders.confluence import ConfluenceLoader
from dotenv import load_dotenv
from typing import (Any,List)

In [2]:
load_dotenv()
persist_directory = "./chroma_db/"
space_key = "IBW"
confluence_url = os.getenv("CONFLUENE_URL")
username = os.getenv("CONFLUENCE_USERNAME")
api_key = os.getenv("CONFLUENCE_API_KEY")

In [3]:
from langchain_community.document_loaders.confluence import ContentFormat

## 1. Extract the documents
loader = ConfluenceLoader(
    #content_format=ContentFormat.STORAGE,
    url=confluence_url,
    username = username,
    api_key= api_key
)

documents = loader.load(
    #space_key=space_key,
    page_ids= [25397420, 25398513], # [26706543, 185500205], # 398098434 # bankens = 25398483, 25398513
    include_attachments=False,
    include_comments=False,
    limit=100,
    max_pages=1000,
    keep_markdown_format=True,
    )

Received runtime arguments {'page_ids': [25397420, 25398513], 'include_attachments': False, 'include_comments': False, 'limit': 100, 'max_pages': 1000, 'keep_markdown_format': True}. Passing runtime args to `load` is deprecated. Please pass arguments during initialization instead.


In [4]:
print("number of document: ", len(documents))

number of document:  2


In [4]:
documents

[Document(page_content='  \n\n\n### **Allmän beskrivning**\n\n\n\n---\n\nICA Banken erbjuder två typer av bankkort för privatkunder, ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard.\xa0Sedan tidigare finns det även något som kallas för migrerat ICA kort. Våra bankkort är gratis de två första månaderna. Det finns ingen övre beloppsgräns från vår sida när kund handlar med korten (undantag gäller vid blipp-köp), så länge kund har det önskade saldot så kan hen handla oavsett summan.\xa0ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard\xa0är öppna för köp över hela världen.  \n\xa0\n\n[Jämförelse av våra kort på hemsidan.](https://www.icabanken.se/kort-och-konto/vara-kort/jamfor-vara-kort/)\n\n### **Bankkort**\n\n\n\n---\n\n* Mastercard Debit\n* Går att ha gemensamt.\n* Innehåller Reseförsäkring och Matförsäkring.\n* Gratis för Stammis Studenter och personer under 21 år.\n* Grundkriterier för att ha möjlighet att bli beviljad kortet: fyllt 16 år (för yngre se sida: ) och bos

## Using Langchain's Recursive and Markdown splitter
Split documents and create text snippets based on `\n, \n\n`

CharacterTextSplitters (both recursive and plain) will merge multiple small tables into 1.


In [307]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts_recursive = text_splitter.split_documents(documents)
texts_recursive


[Document(page_content='| **temp**  | **a**  | **b** |\n| --- | --- | --- |\n| 1 | 2 | 3 |\n| 4 | 5 | 6 |', metadata={'title': 'Test page with one table', 'id': '661258242', 'source': 'https://ica.atlassian.net/wiki/spaces/BOI/pages/661258242/Test+page+with+one+table', 'when': '2024-03-26T07:33:17.802Z'})]

In [45]:
print("total_chunks: ", len(texts_recursive))
print("Length of each chunk: ", [len(x.page_content) for x in texts_recursive])

total_chunks:  8
Length of each chunk:  [831, 999, 1000, 1000, 743, 12, 863, 172]


In [46]:
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)
texts_md = text_splitter.split_documents(documents)
texts_md

[Document(page_content='## \xa0Objective\n\nThe goal is to build a next generation Chatbot based on LLMs for the bank which replaces the current Chatbot that is based on hard-coded workflows.\n\n##  Business Value\n\nPotential business value from improved Chatbot\u200b\n\n* Improving 1st call resolutions???\n* Improve availability of customer service -> 24/7\u200b\n* Improve customer satisfaction\n* Minimizing NVA calls (Enabling customer service agents to work with more high-value calls\u200b)\n* To reduce the overhead of maintaining the "scripts" [scripts are the current static workflows through which a user query passes to get an answer].\n\n## \xa0Success metrics\n\nFollowing are some of the success metrics. [More ca]\n\n\n\n| **Goal** | **Metric** |\n| --- | --- |\n| Improve Customer SatisfactionThis is after newlines | Customer satisfaction score increases |\n|  |  |', metadata={'title': 'BankGPT - Customer Facing', 'id': '562233517', 'source': 'https://ica.atlassian.net/wiki/spa

In [47]:
print("total_chunks: ", len(texts_md))
print("Length of each chunk: ", [len(x.page_content) for x in texts_md])

total_chunks:  9
Length of each chunk:  [817, 12, 999, 1000, 1000, 743, 12, 377, 660]


In [48]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts_text = text_splitter.split_documents(documents)
texts_text

Created a chunk of size 3756, which is longer than the specified 1000


[Document(page_content='## \xa0Objective\n\nThe goal is to build a next generation Chatbot based on LLMs for the bank which replaces the current Chatbot that is based on hard-coded workflows.\n\n##  Business Value\n\nPotential business value from improved Chatbot\u200b\n\n* Improving 1st call resolutions???\n* Improve availability of customer service -> 24/7\u200b\n* Improve customer satisfaction\n* Minimizing NVA calls (Enabling customer service agents to work with more high-value calls\u200b)\n* To reduce the overhead of maintaining the "scripts" [scripts are the current static workflows through which a user query passes to get an answer].\n\n## \xa0Success metrics\n\nFollowing are some of the success metrics. [More ca]\n\n| **Goal** | **Metric** |\n| --- | --- |\n| Improve Customer SatisfactionThis is after newlines | Customer satisfaction score increases |\n|  |  |\n\n## \xa0Timeline', metadata={'title': 'BankGPT - Customer Facing', 'id': '562233517', 'source': 'https://ica.atlassi

In [49]:
print("total_chunks: ", len(texts_text))
print("Length of each chunk: ", [len(x.page_content) for x in texts_text])

total_chunks:  4
Length of each chunk:  [829, 3756, 859, 172]


In [50]:
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0, encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
texts_token = text_splitter.split_documents(documents)
texts_token

[Document(page_content='## \xa0Objective\n\nThe goal is to build a next generation Chatbot based on LLMs for the bank which replaces the current Chatbot that is based on hard-coded workflows.\n\n##  Business Value\n\nPotential business value from improved Chatbot\u200b\n\n* Improving 1st call resolutions???\n* Improve availability of customer service -> 24/7\u200b\n* Improve customer satisfaction\n* Minimizing NVA calls (Enabling customer service agents to work with more high-value calls\u200b)\n* To reduce the overhead of maintaining the "scripts" [scripts are the current static workflows through which a user query passes to get an answer].\n\n## \xa0Success metrics\n\nFollowing are some of the success metrics. [More ca]\n\n\n\n| **Goal** | **Metric** |\n| --- | --- |\n| Improve Customer SatisfactionThis is after newlines | Customer satisfaction score increases |\n|  |  |\n\n## \xa0Timeline\n\ntrue%7B%22title%22%3A%22Roadmap%20Planner%22%2C%22timeline%22%3A%7B%22startDate%22%3A%222024

In [51]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0, encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
texts = text_splitter.split_documents(texts)
texts

Created a chunk of size 3756, which is longer than the specified 1000


[Document(page_content='## \xa0Objective\n\nThe goal is to build a next generation Chatbot based on LLMs for the bank which replaces the current Chatbot that is based on hard-coded workflows.\n\n##  Business Value\n\nPotential business value from improved Chatbot\u200b\n\n* Improving 1st call resolutions???\n* Improve availability of customer service -> 24/7\u200b\n* Improve customer satisfaction\n* Minimizing NVA calls (Enabling customer service agents to work with more high-value calls\u200b)\n* To reduce the overhead of maintaining the "scripts" [scripts are the current static workflows through which a user query passes to get an answer].\n\n## \xa0Success metrics\n\nFollowing are some of the success metrics. [More ca]\n\n| **Goal** | **Metric** |\n| --- | --- |\n| Improve Customer SatisfactionThis is after newlines | Customer satisfaction score increases |\n|  |  |\n\n## \xa0Timeline', metadata={'title': 'BankGPT - Customer Facing', 'id': '562233517', 'source': 'https://ica.atlassi

## Extend the RecursiveCharacter Text splitter 
This works on the Markdown text splitter. It splits the documents into Text and Tables separately

In [19]:
def _split_on_separator(text: str, separator: str, keep_separator: bool) -> List[str]:
    # Now that we have the separator, split the text
    if separator:
        if keep_separator:
            # The parentheses in the pattern keep the delimiters in the result.
            _splits = re.split(f"({separator})", text)
            splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]
            if len(_splits) % 2 == 0:
                splits += _splits[-1:]
            splits = [_splits[0]] + splits
        else:
            splits = text.split(separator)
    else:
        splits = list(text)
    return [s for s in splits if s != ""]

class MarkdownTextTableSplitter(MarkdownTextSplitter):
    """Attempts to split the text along Markdown-formatted headings."""

    def __init__(self, **kwargs: Any):
        """Initialize a MarkdownTextSplitter."""
        super().__init__(**kwargs)
    
    def _split_text(self, text: str, separators: List[str]) -> List[str]: 
     
        """Split incoming text and return chunks."""

        final_chunks = []
        # Get appropriate separator to use
        separator = separators[-1]
        new_separators = None
        for i, _s in enumerate(separators):
            if _s == "":
                separator = _s
                break
            if _s in text:
                separator = _s
                new_separators = separators[i + 1 :]
                break
        
        splits, table_chunks = self._split_tables_and_text(text, separator, self._keep_separator)

       
        # Now go merging things, recursively splitting longer texts.
        _good_splits = []
        _separator = "" if self._keep_separator else separator
        for s in splits:
            if self._length_function(s) < self._chunk_size:
                _good_splits.append(s)
            else:
                if _good_splits:
                    merged_text = self._merge_splits(_good_splits, _separator)
                    final_chunks.extend(merged_text)
                    _good_splits = []
                if new_separators is None:
                    final_chunks.append(s)
                else:
                    
                    other_info = self._split_text(s, new_separators)
                    final_chunks.extend(other_info)
        
        
        if _good_splits:
            merged_text = self._merge_splits(_good_splits, _separator)
            final_chunks.extend(merged_text)
        

        final_chunks.extend(table_chunks)

        clean_chunks = []
        for fc in final_chunks:
            clean_chunks.append(self._replace_newline_with_space(fc))
        
        return clean_chunks

    def _split_tables_and_text(self, text: str, separator: str, keep_separator: bool, table_separator: str = '|'):
        
        _intermediate_splits = _split_on_separator(text, separator, keep_separator)
        
        print(repr(separator), _intermediate_splits)
        all_table_chunks = []
        splits = []
        for item in _intermediate_splits:
            stripped = item.strip()
            if stripped:                                                            
                text_chunks, table_chunks = self._split_tables(stripped, table_separator=table_separator)
                if table_chunks:
                    all_table_chunks.extend(table_chunks)
                splits.append(text_chunks)
                        
        return splits, all_table_chunks
    
    def _split_tables(self, text: str, table_separator: str = '|'):
        table_chunks = {}
        text_chunks  = []
        table_num = 0
        #print(text)
        #print(text.splitlines())
        for line in text.splitlines():
            if line.strip().startswith(table_separator):
                if table_num in table_chunks.keys():
                    table_chunks[table_num].append(line.strip()) 
                else:
                    table_chunks[table_num]=[line.strip()] 
            else:
                table_num +=1
                text_chunks.append(line.strip())
        if table_chunks:
            final_tables = ["\n".join(table_chunks[i]) for i in table_chunks.keys()]
        else:
            final_tables = []

        return "\n".join(text_chunks), final_tables
    
        
    def _replace_newline_with_space(self, text):
        # Define regular expression pattern to match newline character
        newline_pattern = r'\n*'

        # Replace newline characters with space
        text_with_spaces = re.sub(newline_pattern, '', text)

        return text_with_spaces
    


In [16]:
# d = documents[:1]
text_splitter = MarkdownTextTableSplitter(
    #model_name='gpt-4',
    chunk_size=1000, chunk_overlap=0
    )
texts_tb_1 = text_splitter.split_documents(documents)
[f"{len(x.page_content)} : {x.page_content}" for x in texts_tb_1]

['\n#{1,6} ', '```\n', '\n\\*\\*\\*+\n', '\n---+\n', '\n___+\n', '\n\n', '\n', ' ', '']
'\n\n' ['  ', '\n\n\n### **Allmän beskrivning**', '\n\n', '\n\n---', '\n\nICA Banken erbjuder två typer av bankkort för privatkunder, ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard.\xa0Sedan tidigare finns det även något som kallas för migrerat ICA kort. Våra bankkort är gratis de två första månaderna. Det finns ingen övre beloppsgräns från vår sida när kund handlar med korten (undantag gäller vid blipp-köp), så länge kund har det önskade saldot så kan hen handla oavsett summan.\xa0ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard\xa0är öppna för köp över hela världen.  \n\xa0', '\n\n[Jämförelse av våra kort på hemsidan.](https://www.icabanken.se/kort-och-konto/vara-kort/jamfor-vara-kort/)', '\n\n### **Bankkort**', '\n\n', '\n\n---', '\n\n* Mastercard Debit\n* Går att ha gemensamt.\n* Innehåller Reseförsäkring och Matförsäkring.\n* Gratis för Stammis Studenter och personer under 21 

['664 : ### **Allmän beskrivning**---ICA Banken erbjuder två typer av bankkort för privatkunder, ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard.\xa0Sedan tidigare finns det även något som kallas för migrerat ICA kort. Våra bankkort är gratis de två första månaderna. Det finns ingen övre beloppsgräns från vår sida när kund handlar med korten (undantag gäller vid blipp-köp), så länge kund har det önskade saldot så kan hen handla oavsett summan.\xa0ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard\xa0är öppna för köp över hela världen.[Jämförelse av våra kort på hemsidan.](https://www.icabanken.se/kort-och-konto/vara-kort/jamfor-vara-kort/)### **Bankkort**---',
 '511 : * Mastercard Debit* Går att ha gemensamt.* Innehåller Reseförsäkring och Matförsäkring.* Gratis för Stammis Studenter och personer under 21 år.* Grundkriterier för att ha möjlighet att bli beviljad kortet: fyllt 16 år (för yngre se sida: ) och bosatt inom EU/EEA.* Månadsavgift/Cash Advance/Valutaväxlingspås

In [51]:
print("Total chunks: ", len(texts_tb_1))
print("Chunks > 1000 chars: ", len([y for y in [len(x.page_content) for x in texts_tb_1] if y > 1000]))
print("Chunks < 500 chars: ", len([y for y in [len(x.page_content) for x in texts_tb_1] if y < 500]))

Total chunks:  3
Chunks > 1000 chars:  1
Chunks < 500 chars:  0


## Extend the MarkdownHeaderTextSplitter

In [7]:
from copy import deepcopy
from langchain_text_splitters.markdown import MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain_core.documents import Document
import sys


def _split_tables_and_text(doc: str, table_separator: str = '|'):
    
    all_table_chunks = []
    all_text_chunks = []
    stripped = doc.page_content.strip()
    if stripped:                                                            
        text_chunks, table_chunks = _split_tables(stripped, table_separator=table_separator)
        if table_chunks:
            all_table_chunks.extend(table_chunks)
        all_text_chunks.append(text_chunks)
                    
    return all_text_chunks, all_table_chunks

def _split_tables(text: str, table_separator: str = '|'):
        table_chunks = {}
        text_chunks  = []
        table_num = 0
        for line in text.splitlines():
            if line.strip().startswith(table_separator):
                if table_num in table_chunks.keys():
                    table_chunks[table_num].append(line.strip()) 
                else:
                    table_chunks[table_num]=[line.strip()] 
            else:
                table_num +=1
                text_chunks.append(line.strip())
        if table_chunks:
            final_tables = ["\n".join(table_chunks[i]) for i in table_chunks.keys()]
        else:
            final_tables = []

        return "\n".join(text_chunks), final_tables

def process_header_splitted_docs(documents, page_metadata, chunk_size, all_docs):
    all_docs = all_docs or []
    for item in documents:
        subtitle = list(item.metadata.values())[0]
        #print(f"{len(item.page_content)}: ", item.page_content[:30])
        if len(item.page_content) <= chunk_size:
            print("Iam here")
            item.page_content = subtitle + " " + item.page_content
            item.metadata = deepcopy(page_metadata)
            item.metadata["sub-title"] = subtitle
            #print(list(item.metadata.values())[0])
            #print(item.metadata)
            all_docs.append(item)
        else:
            new_docs = text_splitter.split_documents([item])
            process_header_splitted_docs(new_docs, page_metadata, chunk_size, all_docs)
    return all_docs


def process_header_splitted_document(item, metadata, chunk_size, all_docs):
    all_docs = all_docs or []
    
    print(item.metadata)
    #print(f"{len(item.page_content)}: ", item.page_content[:30])
    if len(item.page_content) <= chunk_size:
        item.page_content = list(item.metadata.values())[0] + " " + item.page_content
        item.metadata = deepcopy(metadata)
        item.metadata["sub-title"] = list(item.metadata.values())[0]
        #print(list(item.metadata.values())[0])
        #print(item.metadata)
        all_docs.append(item)
    else:
        new_docs = text_splitter.split_documents([item])
        #print([len(x.page_content) for x in new_docs])
        process_header_splitted_docs(new_docs, item.metadata, chunk_size, all_docs)
    return all_docs



In [45]:
#all_docs = []
chunk_size = 1000
header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "level_1"), ("##", "level_2"), ('###', "level_3")]
)
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)

final = []
for doc in documents:
    print("~"*50)
    
    docs_splitted_by_header = header_splitter.split_text(doc.page_content)

    # for md in docs_splitted_by_header:
    #     print(md.metadata)
    #     print("\n")
   
    text_documents = []
    table_documents = []
    i = 0
    for header_doc in docs_splitted_by_header:

        #print(header_doc.metadata)
        text, table = _split_tables_and_text(header_doc)
        # for md in text:
        #     print("TEXT: ", md)
        # print(len(text_documents))
        if text: 
            text_documents.append(Document(page_content=text[0], metadata=header_doc.metadata))

        if table:
            metadata = deepcopy(doc.metadata)
            metadata["sub-title"] = list(header_doc.metadata.values())[0]
            table_documents.append(
                Document(
                    page_content=list(header_doc.metadata.values())[0] + " " + table[0],
                    metadata=metadata
                )
            )
    
  
    final.extend(process_header_splitted_docs(text_documents, doc.metadata, chunk_size, list()))
    final.extend(table_documents)





~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Iam here
Iam here
Iam here
Iam here
Iam here
Iam here
Iam here
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Iam here
Iam here
Iam here
Iam here


In [42]:
#[f"{len(x.page_content)}: {x.metadata.get('title')} : {x.page_content[:30]}" for x in final]

In [48]:
[(x.page_content[:30], x.metadata["sub-title"]) for x in final]

[('**Allmän beskrivning** ---\nICA', '**Allmän beskrivning**'),
 ('**Bankkort** ---\n* Mastercard ', '**Bankkort**'),
 ('**Bankkort Plus** ---\n* Master', '**Bankkort Plus**'),
 ('**Bankkort Plus** **\\***För me', '**Bankkort Plus**'),
 ('**Migrerat ICA Kort** ---\nMigr', '**Migrerat ICA Kort**'),
 ('**Migrerat ICA Kort** 3. Klick', '**Migrerat ICA Kort**'),
 ('**Migrerat ICA Kort** Lag (201', '**Migrerat ICA Kort**'),
 ('**Bankkort Plus** | Direktkont', '**Bankkort Plus**'),
 ('**Allmän beskrivning** ---\nICA', '**Allmän beskrivning**'),
 ('**Kriterier för att kunna regi',
  '**Kriterier för att kunna registrera sig som familj**'),
 ('**Borttag av familjemedlem** -', '**Borttag av familjemedlem**'),
 ('**Avslut familjepaketet** ---\n', '**Avslut familjepaketet**')]

In [49]:
final

[Document(page_content='**Allmän beskrivning** ---\nICA Banken erbjuder två typer av bankkort för privatkunder, ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard.\xa0Sedan tidigare finns det även något som kallas för migrerat ICA kort. Våra bankkort är gratis de två första månaderna. Det finns ingen övre beloppsgräns från vår sida när kund handlar med korten (undantag gäller vid blipp-köp), så länge kund har det önskade saldot så kan hen handla oavsett summan.\xa0ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard\xa0är öppna för köp över hela världen.\n[Jämförelse av våra kort på hemsidan.](https://www.icabanken.se/kort-och-konto/vara-kort/jamfor-vara-kort/)', metadata={'title': 'Bankkort, Bankkort Plus och Migrerat ICA kort', 'id': '25397420', 'source': 'https://ica.atlassian.net/wiki/spaces/IBW/pages/25397420/Bankkort%2C+Bankkort+Plus+och+Migrerat+ICA+kort', 'when': '2024-03-19T14:27:11.684Z', 'sub-title': '**Allmän beskrivning**'}),
 Document(page_content='**Bankkort** 

In [4]:
# -----------

In [4]:
from splitters_confluence import ConfluenceMarkdownChunker
from langchain_text_splitters.markdown import MarkdownTextSplitter, MarkdownHeaderTextSplitter

chunk_size = 1000
header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "level_1"), ("##", "level_2"), ('###', "level_3")]
)
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)

chunker = ConfluenceMarkdownChunker(markdown_header_splitter=header_splitter, markdown_text_splitter=text_splitter)

In [5]:
final = chunker.chunker(documents)

In [6]:
[f"{x.page_content[:30]} -- {x.metadata['sub-title']}"  for x in final]

['**Allmän beskrivning** ---\nICA -- **Allmän beskrivning**',
 '**Bankkort** ---\n* Mastercard  -- **Bankkort**',
 '**Bankkort Plus** ---\n* Master -- **Bankkort Plus**',
 '**Bankkort Plus** **\\***För me -- **Bankkort Plus**',
 '**Migrerat ICA Kort** ---\nMigr -- **Migrerat ICA Kort**',
 '**Migrerat ICA Kort** 3. Klick -- **Migrerat ICA Kort**',
 '**Migrerat ICA Kort** Lag (201 -- **Migrerat ICA Kort**',
 '**Bankkort Plus** | Direktkont -- **Bankkort Plus**',
 '**Allmän beskrivning** ---\nICA -- **Allmän beskrivning**',
 '**Kriterier för att kunna regi -- **Kriterier för att kunna registrera sig som familj**',
 '**Borttag av familjemedlem** - -- **Borttag av familjemedlem**',
 '**Avslut familjepaketet** ---\n -- **Avslut familjepaketet**']

## Semantic text splitter

In [261]:
from semantic_text_splitter import TextSplitter, MarkdownSplitter
from tokenizers import Tokenizer

# Maximum number of tokens in a chunk
max_tokens = 1000
tokenizer = Tokenizer.from_pretrained("bert-base-uncased") # "google-bert/bert-base-uncased"
splitter = TextSplitter.from_huggingface_tokenizer(tokenizer)

#splitter = TextSplitter.from_tiktoken_model("gpt-3.5-turbo")

chunks = splitter.chunks(documents[0].page_content, max_tokens)


/Users/hle254/anaconda3/envs/confluence/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [265]:
for i, t in enumerate(chunks):
    print(i, ": ", t)
    print("-*"*50)

0 :  ##  Objective

The goal is to build a next generation Chatbot based on LLMs for the bank which replaces the current Chatbot that is based on hard-coded workflows.

##  Business Value

Potential business value from improved Chatbot​

* Improving 1st call resolutions???
* Improve availability of customer service -> 24/7​
* Improve customer satisfaction
* Minimizing NVA calls (Enabling customer service agents to work with more high-value calls​)
* To reduce the overhead of maintaining the "scripts" [scripts are the current static workflows through which a user query passes to get an answer].

##  Success metrics

Following are some of the success metrics. [More ca]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
1 :  | **Goal** | **Metric** |
| --- | --- |
| Improve Customer Satisfaction|This is after newlines| | Customer satisfaction score increases |
| | |  |

##  Timeline
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-

In [263]:
print("Total chunks: ", len(chunks))
print("Chunks > 1000 chars: ", len([y for y in [len(x) for x in chunks] if y > 1000]))
print("Chunks < 500 chars: ", len([y for y in [len(x) for x in chunks] if y < 500]))

Total chunks:  9
Chunks > 1000 chars:  3
Chunks < 500 chars:  3


In [264]:
[len(x) for x in chunks]

[670, 160, 491, 1579, 25, 1594, 934, 1938, 669]

## TableTextSplitter
Extend the CharacterTextSplitter

In [20]:
import re
from typing import (
    Any,
    List,
)

def _split_text(text: str, separator: str, keep_separator: bool) -> List[str]:
    # Now that we have the separator, split the text
    if separator:
        if keep_separator:
            # The parentheses in the pattern keep the delimiters in the result.
            _splits = re.split(f"({separator})", text)
            splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]
            if len(_splits) % 2 == 0:
                splits += _splits[-1:]
            splits = [_splits[0]] + splits
        else:
            splits = text.split(separator)
    else:
        splits = list(text)
    return [s for s in splits if s != ""]

class MarkdownTextTableSplitter2(CharacterTextSplitter):
    """Attempts to split the text along Markdown-formatted headings."""

    def __init__(self, separator: str = "\n\n", **kwargs: Any):
        """Create a new TextSplitter."""
        super().__init__(**kwargs)
        self._separator = separator

    def split_text(self, text: str) -> List[str]:
        """Split incoming text and return chunks."""
        #print(text)
        texts, tables = self._separate_tables_text(text)
        # First we naively split the large input into a bunch of smaller ones.
        splits = _split_text(texts, self._separator, self._keep_separator)
        _separator = "" if self._keep_separator else self._separator
        merged_splits = self._merge_splits(splits, _separator)
        merged_splits.extend(tables)
        return merged_splits

    def _separate_tables_text(self, text: str, table_separator: str = '|'):
        table_chunks = {}
        text_chunks  = []
        table_num = 0

        for line in text.splitlines():
            stripped_line = line.strip()
            if stripped_line:
                if stripped_line.startswith(table_separator):
                    if table_num in table_chunks.keys():
                        table_chunks[table_num].append(stripped_line) 
                    else:
                        table_chunks[table_num]=[stripped_line] 
                else:
                    table_num +=1
                    text_chunks.append(stripped_line)
        if table_chunks:
            final_tables = ["\n".join(table_chunks[i]) for i in table_chunks.keys()]
        else:
            final_tables = []

        return self._separator.join(text_chunks), final_tables


In [21]:
# d = documents[:1]
text_splitter = MarkdownTextTableSplitter2(chunk_size=1000, chunk_overlap=100)
texts_tb = text_splitter.split_documents(documents)
# [f"{len(x.page_content)} : {x.page_content}" for x in texts_tb]

Created a chunk of size 1432, which is longer than the specified 1000
Created a chunk of size 1093, which is longer than the specified 1000
Created a chunk of size 1048, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1110, which is longer than the specified 1000
Created a chunk of size 1784, which is longer than the specified 1000
Created a chunk of size 1076, which is longer than the specified 1000


In [22]:
print("Total chunks: ", len(texts_tb))
print("Chunks > 1000 chars: ", len([y for y in [len(x.page_content) for x in texts_tb] if y > 1000]))
print("Chunks < 500 chars: ", len([y for y in [len(x.page_content) for x in texts_tb] if y < 500]))

Total chunks:  2409
Chunks > 1000 chars:  50
Chunks < 500 chars:  193


In [23]:
#texts_tb

# **********************

## BankGPT test

In [1]:
import os
from langchain_community.document_loaders.confluence import ConfluenceLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, CharacterTextSplitter, TokenTextSplitter, TextSplitter
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
#from langchain.chat_models import ChatOpenAI
from __future__ import annotations


import re
from typing import (
    Any,
    List,
)
load_dotenv()
persist_directory = "./chroma_db/"
space_key = "IBW"
confluence_url = os.getenv("CONFLUENE_URL")
username = os.getenv("CONFLUENCE_USERNAME")
api_key = os.getenv("CONFLUENCE_API_KEY")

from langchain_community.document_loaders.confluence import ContentFormat
from confluence_chunker.confluence_loader_modified import BankGPTConfluenceLoader
loader = BankGPTConfluenceLoader(
    content_format=ContentFormat.STORAGE,
    url=confluence_url,
    username = username,
    api_key= api_key
)

documents = loader.load(
    #space_key=space_key,
    page_ids= [25397420], # [26706543, 185500205], # 398098434 # bankens = 25398483
    include_attachments=False,
    include_comments=False,
    limit=100,
    max_pages=1000,
    keep_markdown_format=True, 
    )

Received runtime arguments {'page_ids': [25397420], 'include_attachments': False, 'include_comments': False, 'limit': 100, 'max_pages': 1000, 'keep_markdown_format': True}. Passing runtime args to `load` is deprecated. Please pass arguments during initialization instead.


In [2]:
print(documents)

['  \n\n\n### **Allmän beskrivning** ###\n\n\n\n---\n\nICA Banken erbjuder två typer av bankkort för privatkunder, ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard.\xa0Sedan tidigare finns det även något som kallas för migrerat ICA kort. Våra bankkort är gratis de två första månaderna. Det finns ingen övre beloppsgräns från vår sida när kund handlar med korten (undantag gäller vid blipp-köp), så länge kund har det önskade saldot så kan hen handla oavsett summan.\xa0ICA Bankkort MasterCard och ICA Bankkort Plus MasterCard\xa0är öppna för köp över hela världen.  \n\xa0\n\n[Jämförelse av våra kort på hemsidan.](https://www.icabanken.se/kort-och-konto/vara-kort/jamfor-vara-kort/)\n\n### **Bankkort** ###\n\n\n\n---\n\n* Mastercard Debit\n* Går att ha gemensamt.\n* Innehåller Reseförsäkring och Matförsäkring.\n* Gratis för Stammis Studenter och personer under 21 år.\n* Grundkriterier för att ha möjlighet att bli beviljad kortet: fyllt 16 år (för yngre se sida: ) och bosatt inom EU/EE

# *****************************************

In [260]:
len(texts_tb), len(texts_tb_1)

(10, 16)

In [28]:
import bs4
from langchain.schema import Document

from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title


class HTMLSplitter:
    """
    The class that splits HTML into chunks of the predefined size `chunk_size`.
    The chunks can overlap by at most `chunk_overlap` characters, meaning that if `chunk_overlap`
    is non-zero, the next chunk should start by at most `chunk_overlap` characters from the last chunk.

    Forming the chunks and calculating their overlap is governed by the following rules:
    1. Do not split in the middle any unbreakable tags
    2. Do not split any HTML tag name, e.g. <a> should never be split into ['<a', '>']
    3. Do not split regular words inside tags in the middle, instead take the whole word until the closest space.

    The unbreakable tags include:
    1. Headers, like <h1> or <h2>
    2. Table rows <tr>

    This should be achieved by `split_html` method. Test cases for `split_html` method:

    Input: <h1>Title</h1><p>Some <b>bold</b> text.</p><tr><td>Table cell</td></tr>
    Output: ['<h1>Title</h1>', '<p>Some ', 'Some <b>bold', 'bold</b> text.', 'text.</p><tr><td>Table cell</td></tr>']
    """

    def __init__(self, chunk_size, chunk_overlap):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.unbreakable_tags = ["h1", "h2", "tr"]
        self.chunks = []

    def split_html(self, html):
        soup = bs4.BeautifulSoup(html, features="html.parser")
        self._split_node(soup)
        return self.chunks

    def split_text(self, text):
        """
        The method takes HTML text as input, splits it according to the rules of this class.
        Then the method converts table into a text representation
        """

        # This is for compatibility with LangChain's interface
        chunks = self.split_html(text)

        return [Document(page_content=c) for c in chunks]

    def _split_node(self, node, stack_before=None, stack_after=None):
        if isinstance(node, bs4.NavigableString):
            self._split_text(
                str(node), stack_before=stack_before, stack_after=stack_after
            )
        elif isinstance(node, bs4.Comment):
            pass  # ignore comments
        else:
            if stack_before is None:
                stack_before = []
            if stack_after is None:
                stack_after = []

            if node.name in self.unbreakable_tags:
                # TODO: this is wrong logic, they shouldn't be just added to chunks
                self.chunks.append(str(node))
            else:
                N_children = len(node.contents)

                for i, child in enumerate(node.children):
                    new_before = list(stack_before)
                    new_after = list(stack_after)

                    if node.name != "[document]":
                        if i == 0:
                            new_before.append(node.name)

                        if i == N_children - 1:
                            new_after.append(node.name)

                    self._split_node(
                        child, stack_before=new_before, stack_after=new_after
                    )

    def get_clean_suffix(self, text, chunk_size=None, boundary=None):
        """
        The function returns the substring of at most size `chunk_size`
        by following the `text` from end to beginning. When found the position
        for the chunk boundary, the function adjusts the string as follows:
        - If the position is in the middle of the word, move right to the closest space
        - If the position is in the middle of the tag, move right to the corresponding closing tag
        """
        N_text = len(text)
        if chunk_size:
            if N_text <= chunk_size:
                return text, 0

            boundary = N_text - chunk_size

        if text[boundary] == ">":
            boundary += 1
            return text[boundary:], boundary

        # If the boundary is in the middle of a word, move right to the closest space
        while boundary < N_text and text[boundary] not in [" ", ">"]:
            boundary += 1

        # If the boundary is in the middle of a tag, move right to the corresponding closing tag
        if boundary < N_text and text[boundary] == ">":
            while boundary < N_text and text[boundary] != "<":
                boundary += 1

        return text[boundary:], boundary

    def _split_text(self, text, stack_before=None, stack_after=None):
        if stack_before:
            text = "<{0}>{1}".format(stack_before[-1], text)
        if stack_after:
            text = "{1}</{0}>".format(stack_after[-1], text)

        if len(self.chunks) > 0 and len(self.chunks[-1]) < self.chunk_size:
            # If the chunk still has room to chunk_size, pop it from chunk
            # in an attempt to extend
            last_chunk = self.chunks.pop()
            text = "{}{}".format(last_chunk, text)

        overlap = ""
        while len(text) > self.chunk_size:
            # This is when the chunk is larger than the `chunk_size`
            # So we need to split it and store the ready chunk,
            # and leave the smaller overflow part in `text`.
            if len(self.chunks) > 0:
                overlap, _ = self.get_clean_suffix(
                    self.chunks[-1], chunk_size=self.chunk_overlap
                )

            idx = (self.chunk_size - self.chunk_overlap) if overlap else self.chunk_size

            _, idx = self.get_clean_suffix(text, boundary=idx)
            self.chunks.append(overlap + text[:idx])

            text = text[idx:]
            overlap = ""
        print(self.chunks)

        if text.strip():
            # Store the not-yet-grown chunk in self.chunk
            self.chunks.append(text)

In [5]:
from unstructured.partition.md import  partition_md
from unstructured.chunking.title import chunk_by_title

In [37]:
import markdown

# Convert Markdown to HTML with table support
html_content = markdown.markdown(documents[0].page_content, extensions=['markdown.extensions.tables'])

# Print or use the HTML content as needed
print(html_content)

<p>Almost all data used for the model comes from the CRM_DM library connected from the SAS Server.</p>
<p>The models are built on monthly data and hence we use the Kund_historik_månadsvis as the "master" where a lot of customer attributes already exists (historical values as well)</p>
<table>
<thead>
<tr>
<th>Library</th>
<th>Table name</th>
<th>Comments</th>
<th>SAS Program </th>
</tr>
</thead>
<tbody>
<tr>
<td>CRM_DM</td>
<td>Kund_Historik_Månadsvis</td>
<td>"master" table that is used as a starting point </td>
<td>prod_master_customer_abt_grundtabell</td>
</tr>
<tr>
<td>CRM_DM</td>
<td>Kund_RFM_Scoring</td>
<td>an RFM calculation done in 2018 with hard coded values for the 5 groups therein. An assessment was done in 2021 Marchand the groups are still fairly stable, so the decision was made to keep the model as is. Room for improvement here though. The RFM information can be found on T:\Bank\ICA Banken BI\SAS\SAS-information  go to the OneNote called "Åtgärder 2018" and then to 30 Ap

In [41]:
def recursive(func):
    def wrapper(self, soup, recursive_before=False, recursive_after=False):
        def recurse():
            if soup is not None:
                for child in soup.children:
                    if isinstance(child, bs4.Tag):
                        wrapper(
                            self,
                            child,
                            recursive_before=recursive_before,
                            recursive_after=recursive_after,
                        )

        if recursive_before:
            recursive_after = False

        if recursive_before:
            recurse()

        func(
            self,
            soup,
            recursive_before=recursive_before,
            recursive_after=recursive_after,
        )

        if recursive_after:
            recurse()

    return wrapper
class BS4Util:
    @recursive
    def delete_all_attrs(self, soup, **kwargs):
        if hasattr(soup, "attrs"):
            for attribute in list(soup.attrs):
                if attribute != "href":
                    del soup[attribute]

    @recursive
    def clean_href(self, soup, **kwargs):
        if hasattr(soup, "attrs"):
            for attribute in list(soup.attrs):
                if attribute == "href":
                    soup[attribute] = soup[attribute].split("?")[0].strip("/")

    @recursive
    def remove_nested_tags(self, soup, **kwargs):
        for tag in soup.find_all(True):
            if tag.parent and tag.parent.name == tag.name:
                tag.replace_with(*tag.contents)

    @recursive
    def remove_empty_tags(self, soup, **kwargs):
        for tag in soup.find_all(True):
            contents = [
                x
                for x in tag.contents
                if not isinstance(x, bs4.NavigableString) or x.strip()
            ]

            if not contents:
                tag.decompose()

    def remove_table_formatting(self, soup):
        if not isinstance(soup, bs4.BeautifulSoup):
            return

        tag_names = ("td", "th")
        for tag_name in tag_names:
            for tag in soup.find_all(tag_name):
                new_tag = soup.new_tag(tag_name)
                new_tag.string = tag.get_text()

                tag.replace_with(new_tag)


In [42]:

splitter = HTMLSplitter(chunk_size=1000, chunk_overlap=0)

obj = bs4.BeautifulSoup(html_content, r"html.parser")
bs4util = BS4Util()

bs4util.delete_all_attrs(obj, recursive_after=True)
bs4util.remove_nested_tags(obj, recursive_after=True)
bs4util.remove_empty_tags(obj, recursive_before=True)
bs4util.remove_table_formatting(obj)

chunks = splitter.split_html(str(obj))

elements = partition_html(text=str(obj))
chunks2 = chunk_by_title(elements)


[]
[]
[]
[]
[]
[]
['<p>Almost all data used for the model comes from the CRM_DM library connected from the SAS Server.</p>\n<p>The models are built on monthly data and hence we use the Kund_historik_månadsvis as the "master" where a lot of customer attributes already exists (historical values as well)</p>\n<table>\n<thead>\n']
['<p>Almost all data used for the model comes from the CRM_DM library connected from the SAS Server.</p>\n<p>The models are built on monthly data and hence we use the Kund_historik_månadsvis as the "master" where a lot of customer attributes already exists (historical values as well)</p>\n<table>\n<thead>\n']
['<p>Almost all data used for the model comes from the CRM_DM library connected from the SAS Server.</p>\n<p>The models are built on monthly data and hence we use the Kund_historik_månadsvis as the "master" where a lot of customer attributes already exists (historical values as well)</p>\n<table>\n<thead>\n']
['<p>Almost all data used for the model comes fro

# Test Retrieval

In [ ]:
#!pip install --upgrade --quiet  gpt4all 
#!pip install langchain-community
# !pip install transformers sentence-transformers

In [10]:
from langchain_community.embeddings import GPT4AllEmbeddings
#from langchain_community.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [24]:
#embeddings = GPT4AllEmbeddings()
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#### Generate Embeddings and add to chroma store

In [25]:
vectordb = Chroma.from_documents(texts_tb, embeddings)

In [ ]:
# import os
# if persist_directory and os.path.exists(persist_directory):
#     vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
# else:
#     vectordb = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=persist_directory)
    

In [26]:
question = "Spärra bankkort"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='Spärra via internetbanken/ICA Bankens app**Internetbanken:**\n\nKund går in under:\xa0*Min Ekonomi → Säker identifiering → Mobilt BankID → Hantera Mobilt BankID → Hantera Mobilt BankID* (externt fönster) → nu kan kunden välja att spärra BankID:t.\n\n**Iphone-/Ipad-appen**:\n\nKund går in via *Inställningar* och sedan *Hantera Mobilt BankID.*\n\n**Android-appen**:\n\nKund går in via *kugghjulet* uppe till höger och sedan\xa0*Ta bort BankID*.\n\nSpärra via BG Tools1. Öppna länken: <https://bgtools.icabanken.net/bgtools.aspx>\xa0i Google Chrome (tips är att spara den som favorit)\n\n2. Logga in med användarnamn och lösenord.\n\n3. Följande sida öppnas, klicka på **Kör**\xa0vid *BankID Admin*.\n\n4. Nedan sida öppnas, ange kundens personnummer ÅÅÅÅMMDDXXXX, klicka på **Fortsätt**.\n\n5. Du\xa0kommer då in på Bankgirots administratörssida. **Kontrollera att det står rätt namn till kunden!** Spärra kundens BankID genom att klicka på **Spärra**.', metadata={'title': 'M

In [27]:
question = "Får jag skicka en personlig dosa utomlands via fullmakt?"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='* Eventuella utskick som vi behöver göra till utlandet kostar upp till 500 SEK.\n\n* Om ansökan beviljas kommer du få hem kort, kortkod och bankdosa i separata kuvert. Om det blir avslag eller om ansökan behöver kompletteras får du besked om det via brev. (Handläggningstiden varierar beroende på om det behövs kompletteringar, vi kan därför inte säga en viss handläggningstid.)\n\n3. **Skapa ett ärende i Centrica** till *Affärssupport IKK* med underkategori: *Utskick av ansökan* och följande information:\n\n* *Beskrivning*: Kortansökan för utvandrad person med svenskt personnummer.\n\n* Personnumret\n\n* Personens för- och efternamn.\n\n* Adressen dit ansökningsblanketten ska skickas.\n\n**Frågor och svar**\n\n---\n\nVad betyder "utvandrad"?Man är utvandrad om man saknar svensk folkbokföringsadress och har meddelat Skatteverket att man har flyttat till ett annat land.', metadata={'title': 'Kortansökan för utvandrad person med svenskt personnummer', 'id': '25397637